# Logistic Regression

## simple input

In [ ]:
from torch import tensor
from torch import nn
from torch import sigmoid
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# Training data and ground truth
x_data = tensor([[1.0], [2.0], [3.0], [4.0]])
y_data = tensor([[0.], [0.], [1.], [1.]])

In [ ]:
class Model(nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate nn.Linear module
        """
        super(Model, self).__init__()
        self.linear = nn.Linear(1, 1)  # One in and one out

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data.
        """
        y_pred = sigmoid(self.linear(x))
        return y_pred

In [ ]:
# our model
model = Model()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = nn.BCELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(1000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data)

    # Compute and print loss
    loss = criterion(y_pred, y_data)
    if epoch % 100 == 0:
        print(f'Epoch {epoch + 1}/1000 | Loss: {loss.item():.4f}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch 1/1000 | Loss: 1.3503
Epoch 101/1000 | Loss: 0.7784
Epoch 201/1000 | Loss: 0.7337
Epoch 301/1000 | Loss: 0.7017
Epoch 401/1000 | Loss: 0.6720
Epoch 501/1000 | Loss: 0.6443
Epoch 601/1000 | Loss: 0.6184
Epoch 701/1000 | Loss: 0.5943
Epoch 801/1000 | Loss: 0.5718
Epoch 901/1000 | Loss: 0.5508


In [ ]:
# After training
print(f'\nLet\'s predict the hours need to score above 50%\n{"=" * 50}')
hour_var = model(tensor([[1.0]]))
print(
    f'Prediction after 1 hour of training: {hour_var.item():.4f} | Above 50%: {hour_var.item() > 0.5}')
hour_var = model(tensor([[7.0]]))
print(
    f'Prediction after 7 hours of training: {hour_var.item():.4f} | Above 50%: { hour_var.item() > 0.5}')


Let's predict the hours need to score above 50%
Prediction after 1 hour of training: 0.4673 | Above 50%: False
Prediction after 7 hours of training: 0.9340 | Above 50%: True


## multiple inputs

https://pytorch.org/docs/stable/nn.html?highlight=bceloss#torch.nn.BCELoss

https://en.wikipedia.org/wiki/Cross_entropy

In [ ]:
from torch import nn, optim, from_numpy
import numpy as np

In [ ]:
xy = np.loadtxt('./data/diabetes.csv.gz', delimiter=',', dtype=np.float32)
x_data = from_numpy(xy[:, 0:-1])
y_data = from_numpy(xy[:, [-1]])
print(f'X\'s shape: {x_data.shape} | Y\'s shape: {y_data.shape}')

X's shape: torch.Size([759, 8]) | Y's shape: torch.Size([759, 1])


In [ ]:
class Model(nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = nn.Linear(8, 6)
        self.l2 = nn.Linear(6, 4)
        self.l3 = nn.Linear(4, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

In [ ]:
# our model
model = Model()


# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = nn.BCELoss(reduction='mean')  # binary cross entropy
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
# Training loop
for epoch in range(100):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data)

    # Compute and print loss
    loss = criterion(y_pred, y_data)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch + 1}/100 | Loss: {loss.item():.4f}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


Epoch: 1/100 | Loss: 0.6840
Epoch: 11/100 | Loss: 0.6576
Epoch: 21/100 | Loss: 0.6493
Epoch: 31/100 | Loss: 0.6466
Epoch: 41/100 | Loss: 0.6456
Epoch: 51/100 | Loss: 0.6453
Epoch: 61/100 | Loss: 0.6452
Epoch: 71/100 | Loss: 0.6452
Epoch: 81/100 | Loss: 0.6451
Epoch: 91/100 | Loss: 0.6451


In [ ]:
# Since I don't have additional data to text...
var = model(x_data)
print((y_data - var).mean())

tensor(0.0010, grad_fn=<MeanBackward0>)
